In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np

In [2]:
# Load the data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
"""
    To classify images using a reccurent neural network, we consider every image row as a sequence of pixels. 
    Because MNIST image shape is 28 * 28 px, we will then handle 28 sequences of 28 steps for every sample.
"""
# Parameters 
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_inputs = 28 # MNIST data input (img shape: 28 * 28)
n_steps = 28  # time step
n_hidden = 128  # hidden layer num of features
n_classes = 10  # MNIST total classes (0-9 digits)

In [8]:
# tf Graph input
x = tf.placeholder("float", [None, n_steps, n_inputs])
y = tf.placeholder("float", [None, n_classes])

# Define weigthts
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [29]:
def RNN(x, weights, biases):
    """
        Prepare data shape to match 'rnn' function requirements
        Current data input shape: (batch_size, n_steps, n_inputs)
        Required shape: 'n_steps' tensor list of shape (batch_size, n_inputs)
    """
    # Permuting batch_size and n_steps
    x = tf.transpose(x, [1, 0, 2])
    # Reshaping to (n_steps * batch_size, n_inputs)
    x = tf.reshape(x, [-1, n_inputs])
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_inputs)
    x = tf.split(0, n_steps, x)
    
    # Define a lstm cell with tensorflow
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)
    
    # Get lstm cell output
    outputs, states = tf.nn.rnn(lstm_cell, x, dtype=tf.float32)
    
    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [30]:
pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

In [33]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, n_steps, n_inputs))
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_inputs))
    test_label = mnist.test.labels[:test_len]
    print( "Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 1.693063, Training Accuracy= 0.41406
Iter 2560, Minibatch Loss= 1.422429, Training Accuracy= 0.56250
Iter 3840, Minibatch Loss= 1.189692, Training Accuracy= 0.60938
Iter 5120, Minibatch Loss= 0.897376, Training Accuracy= 0.69531
Iter 6400, Minibatch Loss= 0.749273, Training Accuracy= 0.71094
Iter 7680, Minibatch Loss= 0.685864, Training Accuracy= 0.78125
Iter 8960, Minibatch Loss= 0.574905, Training Accuracy= 0.84375
Iter 10240, Minibatch Loss= 0.387475, Training Accuracy= 0.84375
Iter 11520, Minibatch Loss= 0.388653, Training Accuracy= 0.85938
Iter 12800, Minibatch Loss= 0.465894, Training Accuracy= 0.85938
Iter 14080, Minibatch Loss= 0.424895, Training Accuracy= 0.88281
Iter 15360, Minibatch Loss= 0.345580, Training Accuracy= 0.87500
Iter 16640, Minibatch Loss= 0.343607, Training Accuracy= 0.86719
Iter 17920, Minibatch Loss= 0.404035, Training Accuracy= 0.87500
Iter 19200, Minibatch Loss= 0.233593, Training Accuracy= 0.93750
Iter 20480, Minibatch Loss= 0.25